In [1]:
import numpy as np
import pandas as pd
import json
import codecs
import datetime
import os
import sys

In [2]:
sys.path.append('.')

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (24, 16)

In [4]:
import oci
import json
from oci.config import validate_config
from sklearn.covariance import MinCovDet 

In [5]:
config = oci.config.from_file(
        "~/.oci/config",
        "DEFAULT")

In [6]:
namespace = "lrzustouvvrg"
bucket_name = "sdk-initial-data-bucket"


In [7]:
validate_config(config)

In [8]:
object_storage_client = oci.object_storage.ObjectStorageClient(config)
next_starts_with = None
object_list = object_storage_client.list_objects(namespace, bucket_name , fields='name, timeCreated, size', start=next_starts_with)

In [9]:
from vector_creator.score_vectors.score_vectors_assembly import score_vector_from_bucket

In [10]:
res0 = score_vector_from_bucket(object_storage_client, 'photo-gallery', next_starts_with)

-> processed -> 1993
-> data to small to process -> 100
-> processed -> 2038
-> data to small to process -> 81
-> data to small to process -> 105
-> processed -> 618
-> data to small to process -> 142
-> data to small to process -> 0
-> data to small to process -> 34
-> processed -> 276
-> data to small to process -> 98
-> processed -> 369
-> processed -> 201
-> processed -> 873
-> data to small to process -> 2
-> processed -> 363
-> processed -> 299
-> data to small to process -> 108
-> data to small to process -> 28
-> data to small to process -> 153
-> processed -> 1288
-> data to small to process -> 0
-> processed -> 753
-> data to small to process -> 81
-> processed -> 765
-> processed -> 717
-> processed -> 334
-> processed -> 201
-> processed -> 235
-> data to small to process -> 194
-> data to small to process -> 108
-> processed -> 311
-> data to small to process -> 0
-> data to small to process -> 0
-> data to small to process -> 21
-> processed -> 1477
-> data to small to pr

In [11]:
next_starts_with = object_list.data.next_start_with

In [12]:
res1 = score_vector_from_bucket(object_storage_client, 'photo-gallery', next_starts_with)

-> data to small to process -> 1
-> processed -> 506
-> data to small to process -> 1
-> processed -> 555
-> processed -> 1632
-> data to small to process -> 169
-> processed -> 691
-> processed -> 385
-> processed -> 255
-> processed -> 394
-> processed -> 599
-> processed -> 5367
-> data to small to process -> 55
-> processed -> 876
-> processed -> 1758
-> processed -> 631
-> data to small to process -> 151
-> processed -> 1436
-> data to small to process -> 21
-> data to small to process -> 177
-> processed -> 849
-> processed -> 544
-> processed -> 525
-> data to small to process -> 132
-> processed -> 9651
-> processed -> 263
-> processed -> 1205
-> data to small to process -> 55
-> processed -> 288
-> processed -> 362
-> processed -> 222
-> processed -> 975
-> data to small to process -> 0
-> processed -> 14690
-> data to small to process -> 28
-> data to small to process -> 31
-> processed -> 530
-> data to small to process -> 113
-> processed -> 389
-> data to small to process 

In [13]:
res = res0.append(res1)

In [14]:
res.to_csv('csv_files/photogallery_ver3.csv')

In [15]:
res.index.is_unique

True

In [16]:
inxs = res.index.values

In [17]:
len(inxs)

1114

In [18]:
from sklearn import preprocessing

In [19]:
def norm_df_2(df):
    inxs = df.index.values
    std_scale = preprocessing.StandardScaler().fit(df)
    df_std = std_scale.transform(df)
    return pd.DataFrame(df_std, columns=df.columns, index=inxs)

#read the tags from json file for user register non register and create df with tag column
def create_tag_df(json_file_path, column_name): # 'entries.json'
    tag_data = json.load(codecs.open(json_file_path, 'r', 'utf-8-sig'))
    df_tag = pd.DataFrame(tag_data)
    df_tag = df_tag.rename(columns={column_name: ''}) # 'app_userid'
    dftag = df_tag.set_index('')
    return dftag

In [20]:
df_n = norm_df_2(res)

In [21]:
df_n.values[:,4].std()

0.9999999999999996

In [22]:
df_t = create_tag_df('entries.json', 'app_userid')
df_j = df_n.join(df_t)
df_j['User_Register'] = df_j['User_Register'].fillna('false')
df_j.head(5)

,median number photos taken in week - location,mad number photos taken in week - scale,sample weeks ratio,mean number of photos in burst,mean number of bursts a week,median night time number photos,mad night time number photos,sampled week nights ratio,entropy on number of photos taken in night time,ar of num of photos in night time lag-1,...,mean number of photos in burst at weekend,mean number of bursts in weekend,median work days number photos,mad work days number photos,sampled weekdays ratio,entropy on number of photos taken in weekdays,ar of num of photos in weekdays lag-1,mean number of photos in burst at weekdays,mean number of bursts in weekdays,User_Register
005b6c49-c4ed-438b-9092-ad02ed4d672e,-0.558442,-0.591002,-1.212474,-0.206990,-0.565058,-0.437939,-0.581170,-1.136415,1.543751,-0.029442,...,-0.156710,-0.505006,-0.503635,-0.531748,-1.221184,1.347373,-0.063895,-0.199354,-0.566016,false
007a0785-1321-4112-be89-e6549a4725a2,-0.558442,-0.591002,-1.535919,0.167101,-0.392087,-0.437939,-0.581170,-1.526136,-0.367050,-0.027988,...,0.232206,-0.369668,-0.503635,-0.531748,-1.506418,-1.272956,0.090092,0.109905,-0.329473,true
00e37960-250f-4185-94ac-143b319730e3,0.212480,0.122906,1.262221,-0.229955,-0.269052,-0.001723,-0.032351,1.337671,0.084353,-0.031780,...,-0.212723,-0.513600,0.303188,0.030507,1.285547,-0.047476,-0.086659,-0.212117,-0.238581,false
01ae3991-50dc-4a67-9c37-cc88ddea90a2,-0.380537,-0.285041,-0.134023,-0.205198,-0.506754,-0.437939,-0.581170,-0.497352,-0.277000,-0.031119,...,-0.195234,-0.609738,-0.349954,-0.281857,-0.166275,-0.253343,-0.072310,-0.190313,-0.473210,false
01c59096-22f1-40fa-b771-49725b91cc93,-0.143330,-0.183054,1.183485,-0.136035,-0.373829,-0.176209,-0.215291,0.974965,0.461648,-0.031253,...,-0.093405,-0.484700,-0.119434,-0.156911,1.159361,0.434586,-0.069192,-0.121188,-0.321819,false


In [23]:
len(df_j[df_j.User_Register == 'true'])

379

In [33]:
df_j.to_csv('csv_files/ModelFiles/photogallery_with_tag_for_xgboost_Oshik.csv')

In [25]:
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import cycle
import pandas as pd

In [26]:
from vector_creator.stats_models.estimators import *

In [32]:
nz = z_score(res.values)

<a id='setup'></a>
 Setting up your notebook session to access ADB

In [28]:
import os
# Enter connection information and wallet file directory

database_name = "DBlouie7"
wallet_storage_directory = os.path.join(os.path.expanduser("~"),"ADB")

In [29]:
import ads
import configparser
import re
import shutil
from tempfile import NamedTemporaryFile
from zipfile import ZipFile

ads.set_documentation_mode(False)

# Create the wallet directory if missing: 
#os.makedirs(wallet_storage_directory, mode=0o700, exist_ok=True)

wallet_path = os.path.join(wallet_storage_directory, database_name)

# Prepare to store ADB connection information
adb_config = os.path.join(wallet_storage_directory, '.credentials')

# Write a configuration file for login creds.
config = configparser.ConfigParser()
config.read(adb_config)


['/home/datascience/ADB/.credentials']

### Test the connection to the ADB.

The easiest way to test the ADB connection is to run the `sqlplus` CLI tool. The `TNS_ADMIN` variable needs to be set and then the connection string is used to connect. This connection string is of the format `\<user_name>/\<password>@\<SID>`. This information can be obtained from the credentials file.

Once this is working, the notebook will demonstrate other methods to connect that are more practical for use within notebooks.

In [31]:
# Add TNS_ADMIN to the environment
os.environ['TNS_ADMIN'] = config[database_name].get('tns_admin')

# Test the database connection
creds = config[database_name]
connect = 'sqlplus ' + creds.get('user') + '/' + creds.get('password') + '@' + creds.get('sid')
print(os.popen(connect).read())


SQL*Plus: Release 19.0.0.0.0 - Production on Thu Nov 25 12:54:50 2021
Version 19.6.0.0.0

Copyright (c) 1982, 2019, Oracle.  All rights reserved.

Last Successful login time: Wed Nov 24 2021 09:45:04 +00:00

Connected to:
Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production
Version 19.13.0.1.0

SQL> Disconnected from Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production
Version 19.13.0.1.0



In [ ]:
df_j.ads.to_sql(
    "MY_TABLE",
    connection_parameters=connection_parameters,
    if_exists="replace"
)